<a href="https://colab.research.google.com/github/toddpglidden/toddpglidden.github.io/blob/main/roberta_on_airline_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
import datasets
from datasets import load_dataset

In [ ]:
ds = load_dataset("osanseviero/twitter-airline-sentiment")

In [ ]:
ds

In [ ]:
ds = ds['train']

In [ ]:
ds

Dataset({
    features: ['tweet_id', 'airline_sentiment', 'airline_sentiment_confidence', 'negativereason', 'negativereason_confidence', 'airline', 'airline_sentiment_gold', 'name', 'negativereason_gold', 'retweet_count', 'text', 'tweet_coord', 'tweet_created', 'tweet_location', 'user_timezone'],
    num_rows: 14640
})

In [ ]:
ds= ds.select_columns(['text', 'airline_sentiment'])

In [ ]:
ds

Dataset({
    features: ['text', 'airline_sentiment'],
    num_rows: 14640
})

In [ ]:
ds = ds.rename_column('airline_sentiment', 'label')

In [ ]:
ds

Dataset({
    features: ['text', 'label'],
    num_rows: 14640
})

In [ ]:
from transformers import pipeline

pipe = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-sentiment-latest")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
outputs = pipe(ds['text'])

In [ ]:
outputs

[{'label': 'neutral', 'score': 0.8863000869750977},
 {'label': 'negative', 'score': 0.9211513996124268},
 {'label': 'negative', 'score': 0.6412557363510132},
 {'label': 'negative', 'score': 0.914168119430542},
 {'label': 'negative', 'score': 0.938209056854248},
 {'label': 'negative', 'score': 0.8962817192077637},
 {'label': 'negative', 'score': 0.7614868879318237},
 {'label': 'negative', 'score': 0.8952987194061279},
 {'label': 'positive', 'score': 0.9165878891944885},
 {'label': 'positive', 'score': 0.9892332553863525},
 {'label': 'negative', 'score': 0.8738214373588562},
 {'label': 'positive', 'score': 0.9841437339782715},
 {'label': 'positive', 'score': 0.985805332660675},
 {'label': 'positive', 'score': 0.9696857333183289},
 {'label': 'positive', 'score': 0.6961560845375061},
 {'label': 'negative', 'score': 0.612587571144104},
 {'label': 'positive', 'score': 0.9915081262588501},
 {'label': 'negative', 'score': 0.8010463714599609},
 {'label': 'positive', 'score': 0.9881396293640137}

In [ ]:
actual = ds['label']

In [ ]:
type(actual)

list

In [ ]:
actual[0]

'neutral'

In [ ]:
outputs[0]['label']

'neutral'

In [ ]:
correct = 0
total = 0
for i, label in enumerate(outputs):
    if label['label'] == actual[i]:
        correct += 1
    total += 1

In [ ]:
print(f"Accuracy: {correct/total*100}")

Accuracy: 81.00409836065575
